## Example

### Definicion de funciones necesarias para generacion malla

In [12]:
%config IPCompleter.greedy=True
import numpy as np
import os
import bempp.api
from mesh_converter import *
from derivate_operators import *

### Seleccion de proteina a simular

In [14]:
directory = '/home/ian/Desktop/Forces_bioelectrostatics'
protein = '1lyz'
forcefield = 'amber'
dir_prot = directory+'/pqr_files/'+protein
pf = protein +'_' + forcefield
density = 0.5
probe_radius = 1.4

convert_pqr2xyzr('{}/{}.pqr'.format(dir_prot,pf),'{}/{}.xyzr'.format(dir_prot,pf))
generate_nanoshaper_mesh('{}/{}.xyzr'.format(dir_prot,pf), dir_prot,pf,density,probe_radius,False)
grid = import_msms_mesh('{}/{}.face'.format(dir_prot,pf),'{}/{}.vert'.format(dir_prot,pf))

#grid = bempp.api.import_grid('5pti_d1.msh')


In [16]:
import bempp.api
#bempp.api.set_ipython_notebook_viewer()
bempp.api.PLOT_BACKEND = "gmsh"



grid.plot()

<img src="5pti_mesh.png", style="width:400px">

and read the `.pqr` file for the charges.

In [20]:
import numpy as np
q, x_q = np.array([]), np.empty((0,3))

#ep_in = 4.
ep_in = 2.
ep_ex = 80.
k = 0.125

# Read charges and coordinates from the .pqr file
molecule_file = open('{}/{}.pqr'.format(dir_prot,pf), 'r').read().split('\n')
for line in molecule_file:
    line = line.split()
    if len(line)==0 or line[0]!='ATOM': continue
    q = np.append( q, float(line[8]))
    x_q = np.vstack(( x_q, np.array(line[5:8]).astype(float) ))

### Right-hand side

With that, we can compute the potential due to the charges on the boundary, required for the right-hand side

In [21]:
# Function to calculate the potential by charges at boundary

dirichl_space = bempp.api.function_space(grid, "DP", 0)
neumann_space = bempp.api.function_space(grid, "DP", 0)

@bempp.api.real_callable
def charges_fun(x, n, domain_index, result):
    global q, x_q, ep_in
    suma = 0
    for k in range(len(q)):
        suma = suma + q[k]/(np.linalg.norm(x-x_q[k]))
    result[:] = suma/(4*np.pi*ep_in)
    #result[:] = np.sum(q / np.linalg.norm(x - x_q, axis=1))
    
charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun=charges_fun)
#charged_grid_fun.plot()

rhs = np.concatenate([charged_grid_fun.coefficients, 
                      np.zeros(neumann_space.global_dof_count)])


/home/ian/miniconda3/envs/bempp/lib/python3.9/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Operators and Matrix

Next, we generate the $2\times 2$ block matrix with the single and double layer operators of the Laplace and Yukawa kernels 

In [22]:
# Define Operators
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
identity = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
slp_in   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, k)
dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, k)

# Matrix Assembly
blocked = bempp.api.BlockedOperator(2, 2)
blocked[0, 0] = 0.5*identity + dlp_in
blocked[0, 1] = -slp_in
blocked[1, 0] = 0.5*identity - dlp_out
blocked[1, 1] = (ep_in/ep_ex)*slp_out
op_discrete = blocked.strong_form()

### Solver

We now use `gmres` from `scipy` to solve the system. 

In [23]:
import inspect
from scipy.sparse.linalg import gmres

array_it, array_frame, it_count = np.array([]), np.array([]), 0
def iteration_counter(x):
        global array_it, array_frame, it_count
        it_count += 1
        frame = inspect.currentframe().f_back
        array_it = np.append(array_it, it_count)
        array_frame = np.append(array_frame, frame.f_locals["resid"])
        #print "It: {0} Error {1:.2E}".format(it_count, frame.f_locals["resid"])        

x, info = gmres(op_discrete, rhs, callback=iteration_counter, tol=1e-3, maxiter=1000, restart = 2000)

print("The linear system was solved in {0} iterations".format(it_count))

The linear system was solved in 64 iterations


The two following `GridFunction` calls store the calculated boundary potential data (separated by $\phi$ and $\frac{\partial \phi}{\partial n}$) for visualization purposes. 

In [25]:
solution_dirichl = bempp.api.GridFunction(dirichl_space, 
                                          coefficients=x[:dirichl_space.global_dof_count])
solution_neumann = bempp.api.GridFunction(neumann_space, 
                                          coefficients=x[dirichl_space.global_dof_count:])
solution_dirichl.plot()
#solution_neumann.plot()

which should give something like:

<img src="5pti_solu.png", style="width:300px">

### Energy calculaton

To compute $\phi_\text{reac}$ at the atoms locations, we use `operators.potential` to then multiply by the charge and add to compute $\Delta G_\text{solv}$. The `332.064` term is just a unit conversion constant.

In [26]:
slp_q = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
dlp_q = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
phi_q = slp_q*solution_neumann - dlp_q*solution_dirichl




# Calcular phi coulomb

# Reaction force calculation (qE)
h=0.001
grad_phi = solvent_potential_first_derivate(x_q, h, neumann_space, dirichl_space, solution_neumann, solution_dirichl)
F_reac = np.zeros([len(q),3])
for j in range(len(q)):
    F_reac[j,:] = -ep_in*q[j]*grad_phi[j,:]
F_reactotal = np.zeros([3])
for j in range(len(q)):
    F_reactotal[:] = F_reactotal[:] + F_reac[j,:]
F_reactotal[:] = 4*np.pi*332.064*F_reactotal[:]
# Dielectric boundary force calculation (DBF)

# Ionic boundary force calculation (IBF)
    
print("Total reaction force: {:10.6f}{:10.6f}{:10.6f} []".format(F_reactotal[0],F_reactotal[1],F_reactotal[2]))
# total dissolution energy applying constant to get units [kcal/mol]
total_energy = 2*np.pi*332.064*np.sum(q*phi_q).real
print("Total solvation energy: {:7.2f} [kcal/Mol]".format(total_energy))

Total reaction force: -1483.709695-376.847851-535.874328 []
Total solvation energy: -1500.48 [kcal/Mol]


In [10]:
grid.normals[1]

array([-0.80518988, -0.5887025 ,  0.0714046 ])

In [1]:
solution_neumann.coefficients

NameError: name 'solution_neumann' is not defined

## References
[1] Yoon, B. J., & Lenhoff, A. M. (1990). A boundary element method for molecular electrostatics with electrolyte effects. *Journal of Computational Chemistry*, 11(9), 1080-1086.

[2] Dolinsky, T. J., Nielsen, J. E., McCammon, J. A., & Baker, N. A. (2004). PDB2PQR: an automated pipeline for the setup of Poisson–Boltzmann electrostatics calculations. *Nucleic acids research*, 32(suppl_2), W665-W667.

[3] Sanner, M. F., Olson, A. J., & Spehner, J. C. (1995, September). Fast and robust computation of molecular surfaces. In *Proceedings of the eleventh annual symposium on Computational geometry* (pp. 406-407). ACM.